In [41]:
from nltk import word_tokenize
import threading
from operator import itemgetter
from ecommercetools import seo

In [42]:
def process(content):
    #------- NORMALIZATION ----------------#
    normalize = []
    for line in content:
        normalize.append(' '.join([''.join([ch.lower() for ch in word if ch.isalpha()]) for word in line.split()]))
    #------- TOKENIZATION ----------------#
    tokenize = []
    for line in normalize:
        tokenize.append(nltk.word_tokenize(line))
    #------- STOPWORD REMOVAL ----------------#
    stopword_lst = nltk.corpus.stopwords.words(fileids='english')
    nostop = []
    for line in tokenize:
        nostop.append([word for word in line if word not in stopword_lst])
    #------- Bi-Gram------------------------#
    newData = []
    for i in range(len(nostop)):
        data = []
        for bi in list(nltk.bigrams(nostop[i])):
            ls = list(bi)
            ls.sort()
            data.append(' '.join(ls))
        newData.append(data)
    #------- Word Frequency ----------------#
    wordFreq = {}
    for line in newData:
        for word in line:
            if(word not in wordFreq.keys()):
                wordFreq[word] = 0
            wordFreq[word] += 1
    return wordFreq

In [43]:
def findTopic(question, k = 1):
    try:
        results = seo.get_serps(question)
    except:
        return ("",[])
    wordDict = process(results.title)
    res = dict(sorted(wordDict.items(), key = itemgetter(1), reverse = True)[:k])
    res = list(res.keys())
    res.sort()
    res = ' '.join(res)
    return (res,list(results.link)[:3])

In [44]:
def fetchOnlineAns(paper):
    paper_topics = {}
    topic_qnNo = {}
    def useless_function(key):
        paper_topics[key] = findTopic(paper[key])
        if len(paper_topics[key][1])==0:
            return
        if paper_topics[key][0] not in topic_qnNo.keys():
            topic_qnNo[paper_topics[key][0]] = []
        topic_qnNo[paper_topics[key][0]].append(key)

    threads=[]
    for key in list(paper.keys())[:20]:
        t = threading.Thread(target=useless_function, args=[key])
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    marklist = sorted(topic_qnNo.items(), key=lambda x:len(x[1]), reverse=True)
    sortdict = dict(marklist)
    return {'links':paper_topics,'impQns':sortdict}

In [45]:
with open('paper2.txt') as f:
    data = f.read()
import ast
paper = ast.literal_eval(data)
paper

{'P1.Q1': 'Q.1 Explain the following file allocation methods with suitable diagram(s): (5) (a) Linked allocation (b) Indexed allocation',
 'P1.Q2': 'Q.2 Differentiate between peer-to-peer and client-server computing (5) environments.',
 'P1.Q3': "Q.3 (a) Write and explain the pseudo-code/algorithm/program of (5) Peterson's Solution used for accessing the critical section by two co-operating processes.  (5)",
 'P1.Q4': 'Q.4 On a system with n CPUs, what is the minimum number of processes that (5) can be in the ready, run, and blocked states? Explain with a suitable process state diagram.',
 'P1.Q5': 'Q.5 A system has three processes and four identical resources. Each process (5) needs a maximum of three resources to complete. Is deadlock possible? Explain your answer.',
 'P2.Q1': 'Q.6 Define concurrency and race condition.',
 'P2.Q2': 'Q.7 Consider a computer system with 264bits virtual address space and page (5) size of 1 Kbyte. The physical memory is of 64 Kbytes and the size of page 

In [47]:
%%time
fetchOnlineAns(paper)

CPU times: user 1.49 s, sys: 208 ms, total: 1.7 s
Wall time: 2.92 s


{'links': {'P2.Q1': ('condition race',
   ['https://www.techtarget.com/searchstorage/definition/race-condition',
    'https://www.tutorialspoint.com/race-condition-critical-section-and-semaphore',
    'https://www.oreilly.com/openbook/linuxdrive3/book/ch05.pdf']),
  'P4.Q1': ('concept process',
   ['https://www.quora.com/On-a-system-with-n-CPUs-what-is-the-maximum-number-of-processes-that-can-be-in-the-ready-run-and-blocked-states',
    'https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/5_CPU_Scheduling.html',
    'https://www.cse.iitb.ac.in/~mythili/os/ps/proc/ps-proc.pdf']),
  'P4.Q3': ('control process',
   ['https://www.tutorialspoint.com/what-is-process-control-block-pcb',
    'https://www.geeksforgeeks.org/process-table-and-process-control-block-pcb/',
    'https://www.scaler.com/topics/operating-system/process-control-block-in-os/']),
  'P1.Q3': ('process synchronization',
   ['https://www.geeksforgeeks.org/petersons-algorithm-in-process-synchronization/',
    'https://